In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep # control the crawl rate to avoid hammering the servers with too many requests
from random import randint
import re # regular expression

In [2]:
def parsing_pages(list_of_pages):
    pages = []
    for page in list_of_pages:
        if pd.notnull(page):
            all_pages = requests.get(page)
            each_page = BeautifulSoup(all_pages.content, "html.parser")
            pages.append(each_page)
            sleep(randint(2,6))
        else: 
            pages.append(None)
    return pages

## Scarpping H Mart Grocery Items

In [3]:
hmart = ['https://www.hmart.com/groceries?p=' + str(i) for i in range(1,16)]
hmart_parsed = parsing_pages(hmart)

In [4]:
# finding item names from a list of beautifulsoup objects
key = 'product name product-item-name'
items = [] 
for page in hmart_parsed:
    items_in_pages = page.find_all('strong', class_=key)
    for item in items_in_pages:
        items_per_page = item.get_text().replace('\n','').replace('\r','').strip().replace('     ', ' ').lower()
        items.append(items_per_page)
# a = hmart_parsed[0].find_all('strong', class_=key)
# [i.get_text() for i in a]
brands = [i.split('        ')[0] for i in items] # 8 whitespaces for the best result
item_names = [i.split('        ')[-1] for i in items]

In [5]:
regrex = re.compile(r'\d\..*')
item_volumns = [regrex.findall(i) for i in item_names]
# item_test = []
# item_volumns = []
# for item in item_names:
#     item_volumn = regrex.findall(item)
#     item_test.append(item_volumn)
#     for i in item_volumn:
#         if i != '':
#             item_volumns.append(i)
#         else:
#             item_volumns.append(None)
#             print('No match!')

In [ ]:
## asian items with brand names
asian_items = []
for i, j in zip(brands, item_names):
    asian_item = i + ' ' + j
    asian_items.append(asian_item)

asian_items_no_volumns = []
for i in asian_items:
    asian_item = re.sub(' \d\..*', '', i)
    asian_items_no_volumns.append(asian_item)

# asian_items_no_volumns
# item_names

In [6]:
len(item_volumns)

704

In [244]:
columns = {
#     'brand_name': brands,
    'item_name': asian_items_no_volumns,
    'item_volumn': item_volumns,
    'type': 'asian'
}
asian_items = pd.DataFrame(data = columns)

In [245]:
asian_items.head()

,item_name,item_volumn,type
0,cj hetbahn cooked white rice box,[7.4oz(210g) 12 ea],asian
1,cj cooked white rice with seaweed soup,[5.9oz(167g)],asian
2,samyang hot chicken flavor ramen,[4.94oz(140g) 5 packs],asian
3,cj cooked white rice with soft tofu stew,[6.1oz(174g)],asian
4,haioreum peeled roasted chestnuts,[3.52oz(100g)],asian


In [ ]:
asian_items.to_csv('hmar')

## Scraping Items from Other Supermarket

In [261]:
# walmart = ['https://www.walmart.com/browse/food/976759?page=' + str(i) for i in range(1,26)]
# walmart_parsed = parsing_pages(walmart)

In [270]:
# test = walmart_parsed[0].find_all('div', class_='search-result-product-title gridview')
# [i.find('a').get_text() for i in test] # dynamic websites

['Cheerios, Whole Grain Oats Cereal, Gluten Free, 18 oz',
 '(3 Pack) Lipton Beefy Onion Soup and Dip Mix, 2.2 oz',
 '(12 Count) Gatorade Thirst Quencher Sports Drink, Cool Blue, 12 fl oz',
 '(8 Cans) StarKist Chunk Light Tuna in Water, 5 oz',
 'Premier Protein Shake, Chocolate, 30g Protein, 11 Fl Oz, 4 Ct',
 'eat2explore Educational Food & Culture Explorer Box - Explore China!',
 'My College Crate Microwaveables Ultimate Snack Care Package for College Students - Variety Assortment of Microwaveables, Mac & Cheese, Popcorn, Ramen, Chips, Gummies & Candies (50 Snacks)',
 '(3 Pack) Betty Crocker Hamburger Helper, Double Cheeseburger Macaroni Hamburger Helper, 12.1 Oz Box (Twin Pack)',
 'Doritos Nacho Cheese Flavored Tortilla Chips, Party Size, 15 oz',
 'Kraft Original Flavor Mac and Cheese, 5 ct - 7.25 oz Multipack']

In [10]:
# kroger = ['https://www.kroger.com/pl/packaged-meals-sides/01011?page=' + 
#                     str(i) for i in range(1,13)]
# kroger_test = 'https://www.kroger.com/pl/packaged-meals-sides/01011?page=1'
# test = requests.get(kroger_test)
# test_parsed = BeautifulSoup(test.content, 'html.parser')

# test
# kroger_beverages_parsed = parsing_pages(kroger_beverages)

In [187]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_binary # adds chromedriver binary to path
from selenium.webdriver.common.by import By # for wait time
from selenium.webdriver.support.ui import WebDriverWait # for wait time
from selenium.webdriver.support import expected_conditions as EC # for wait time conditions

driver = webdriver.Chrome()
driver.get("https://www.walmart.com/browse/food/")

In [155]:
items = driver.find_elements_by_css_selector(".product-title-link.line-clamp span")

In [156]:
list_of_items = [i.text for i in items]

In [179]:
# driver.find_element_by_css_selector(".elc-icon.paginator-hairline-btn").click()

In [188]:
import time
from random import randint

list_of_items_from_pages = []
for n in range(1,26):
        driver.get("https://www.walmart.com/browse/food/?page=" + str(n))
        items_from_pages = driver.find_elements_by_css_selector(".product-title-link.line-clamp span")
        for i in items_from_pages:
            items_extracted = i.text
            list_of_items_from_pages.append(items_extracted)
        time.sleep(3)

In [189]:
len(list_of_items_from_pages)

994

In [203]:
walmart = pd.Series(list_of_items_from_pages)
walmart.to_csv('walmart.csv')

### Cleaning the Item Names

In [192]:
# getting rid of units/volumns at the beginning of each item
pack = re.compile(r'\(\d{1,} (?:pack|count|cans)\)') 
# list_of_items = [i.loewr() for i in list_of_items]
list_of_items_from_pages = [re.sub(pack, '', i.lower()) for i in list_of_items_from_pages]

# isolating volumns at the end
regrex = re.compile(r'(?:, \d{1,}|\d{1,} (?:mg|oz|fl|ct)|, \d{1,} (?:mg|oz|fl|ct)).*')
walmart_item_volumns = [regrex.findall(i) for i in list_of_items_from_pages]
walmart_item_names = [re.sub(regrex, '', i.split(',')[0].strip()) for i in list_of_items_from_pages]

In [246]:
columns = {
    'item_name': walmart_item_names,
    'item_volumn': walmart_item_volumns,
    'type': 'non-asian'
}
walmart_items = pd.DataFrame(data = columns)
walmart_items.head()


,item_name,item_volumn,type
0,lipton recipe secrets soup and dip mix onion,[2 oz 2 count],non-asian
1,doritos nacho cheese flavored tortilla chips,"[, 15 oz]",non-asian
2,equate 70% isopropyl alcohol,"[, 32 oz]",non-asian
3,purina dog chow complete with real chicken adu...,[],non-asian
4,assurance premium xl disposable washcloths,"[, 144 ct]",non-asian


## Tokenizing Item Names

In [267]:
## First, concat the two lists together

frames = [walmart_items, asian_items]
df = pd.concat(frames).reset_index(drop = True)


## seperate training and test sets
from random import sample
df_train = df.sample(frac=0.6, random_state=0)
df_test = df.drop(dt_train.index)

In [268]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vect = CountVectorizer(stop_words='english',max_df=0.85)
train_vect = count_vect.fit_transform(df_train.item_name)

In [269]:
train_tf_transformer = TfidfTransformer(use_idf=False).fit(train_vect)
train_tf = train_tf_transformer.transform(train_vect)
train_tf.shape

(1019, 1719)

In [271]:
from sklearn.decomposition import NMF

nmf = NMF(n_components = 20, random_state=3,
          init='nndsvd')# .fit(upc_tf) # nndsvd for sparse data
#          alpha=.1, l1_ratio=.5)
nmf_fit = nmf.fit(train_tf)

In [272]:
test_vect = count_vect.transform(df_test.item_name) # not fitting, but transforming
# test_tf_transformer = TfidfTransformer(use_idf=False).fit(train_vect)
test_tf = train_tf_transformer.transform(test_vect)
test_tf.shape

test_nmf = nmf_fit.transform(test_tf)

In [273]:
from sklearn.linear_model import LogisticRegression

features = 

X_train = train_tf
y_train = df_train.type

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

X_test = test_tf
y_test = df_test.type

log_reg.score(X_test, y_test)

0.946980854197349

## Test with Items at Kroger

In [291]:
driver = webdriver.Chrome()
driver.get("https://www.kroger.com/search?fulfillment=all&page=1&query=food")

In [278]:
test = driver.find_elements_by_css_selector(".kds-Text--m.text-default-800.mt-12.mb-4.font-500")
[i.text for i in test]

In [281]:
# [i.text for i in elements]

In [292]:
# from random import randint

kroger_items = []
for n in range(1,3):
    driver.get("https://www.kroger.com/search?fulfillment=all&page=" + str(n) + "&query=food")
    items = driver.find_elements_by_css_selector(".kds-Text--m.text-default-800.mt-12.mb-4.font-500")
    for i in items:
        items_extracted = i.text
        kroger_items.append(items_extracted)
        time.sleep(randint(3,10))

In [290]:
kroger_items

[]

(Kroger denied access :P)

## Test with Wegmans

In [313]:
# driver = webdriver.Chrome()
# driver.get("https://shop.wegmans.com/shop/categories/283?page=1")
# time.sleep(3)
# driver.find_element_by_css_selector("#shopping-selector-shop-context-intent-instore").click()
# time.sleep(3)
text = driver.find_elements_by_css_selector(".cell-title")


In [320]:
driver = webdriver.Chrome()

wegmans_items = []
driver.get("https://shop.wegmans.com/shop/categories/283?page=1")
time.sleep(3)
driver.find_element_by_css_selector("#shopping-selector-shop-context-intent-instore").click()
time.sleep(3)
for n in range(1,133):
    time.sleep(2)
    driver.get("https://shop.wegmans.com/shop/categories/283?page=" + str(n))
    time.sleep(3)
    items = driver.find_elements_by_css_selector(".cell-title")
    for i in items:
        items_extracted = i.text
        wegmans_items.append(items_extracted)
    time.sleep(randint(3,10))

In [321]:
len(wegmans_items)

6691

In [322]:
wegmans_items = pd.Series(wegmans_items)
wegmans_items.to_csv('wegmans.csv')

In [316]:
testing = [i.text for i in text]
testing[0].split('\n')

['Wegmans Black Beans', '15.5 ounce', '15B']